# Benchmark the CUDA timestep and its components

In [1]:
CODE_ROOT = pwd() * "/"

"/home/rrs/Desktop/MPAS_Ocean/"

In [4]:
using PyPlot, PyCall
animation  = pyimport("matplotlib.animation")
ipydisplay = pyimport("IPython.display")

PyObject <module 'IPython.display' from '/home/rrs/anaconda3/envs/MPAS_Ocean/lib/python3.8/site-packages/IPython/display.py'>

In [5]:
import CUDA

import CUDAKernels

import KernelAbstractions

In [7]:
const MAX_THREADS_PER_BLOCK = CUDA.attribute(
        CUDA.CuDevice(0), CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK,
)

1024

# Load Mesh From File

In [8]:
mpasOcean = MPAS_Ocean(false, CODE_ROOT * "MPAS_O_Shallow_Water/Mesh+Initial_Condition+Registry_Files/Periodic",
                "base_mesh.nc", "mesh.nc", periodicity="Periodic"
)
typeof(mpasOcean)

MPAS_Ocean

# Define Initial Condition

In [9]:
function gaussianInit!(mpasOcean; sx=1/3, sy=1/3, mx=1/2, my=1/2)
    sigmax = mpasOcean.lX*sx
    sigmay = mpasOcean.lY*sy
    
    mux = mpasOcean.lX*mx
    muy = mpasOcean.lY*my
    
    for iCell in 1:mpasOcean.nCells
        mpasOcean.sshCurrent[iCell] = exp( - (mpasOcean.xCell[iCell] - mux)^2 / sigmax^2 - (mpasOcean.yCell[iCell] - muy)^2 / sigmay^2 )
    end
    
    mpasOcean.normalVelocityCurrent .= 0
    
    return nothing
end

gaussianInit! (generic function with 1 method)

In [10]:
function planeWaveInit!(mpasOcean; rkx = 1, rky = 1)
    kx = rkx * 2 * pi / mpasOcean.lX
    ky = rky * 2 * pi / mpasOcean.lY
    
    for iCell in 1:mpasOcean.nCells
        mpasOcean.sshCurrent[iCell] = cos( kx * mpasOcean.xCell[iCell] + ky * mpasOcean.yCell[iCell] )
    end
    
    mpasOcean.normalVelocityCurrent .= 0
    
    mpasOcean.sshTendency[:] .= mpasOcean.sshCurrent[:]
    mpasOcean.normalVelocityTendency[:] .= mpasOcean.normalVelocityCurrent[:]
    
    return nothing
end

planeWaveInit! (generic function with 1 method)

In [11]:
moveArrays!(mpasOcean, Array)

planeWaveInit!(mpasOcean) # could also do gaussianInit!

# Benchmark GPU vs CPU

In [16]:
using BenchmarkTools

In [17]:
moveArrays!(mpasOcean, Array)
gaussianInit!(mpasOcean)
# save gaussian initial condition so it doesn't have to be recalculated every time
initialSSH = mpasOcean.sshCurrent[:]
initialNormalVelocity = mpasOcean.normalVelocityCurrent[:]

7500-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

## whole time step

In [31]:
# benchmark time it takes to do a timestep on the GPU
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark forward_backward_step_cuda!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 1 evaluations.
 Range (min … max):  18.428 μs …  5.874 ms  ┊ GC (min … max): 0.00% … 99.25%
 Time  (median):     40.187 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.742 μs ± 59.536 μs  ┊ GC (mean ± σ):  1.47% ±  0.99%

  ▃▄▂                                  ▁▂▆██▆▂                ▂
  ████▆▄▄▃▁▄▅▄▆▅▆▆▆▄▃▁▁▁▁▁▁▁▆▅▅▅▅▆▆▆▆▆▇████████▇▇▇▇▇▆▆▅▆▅▅▆▅▆ █
  18.4 μs      Histogram: log(frequency) by time      49.8 μs <

 Memory estimate: 8.97 KiB, allocs estimate: 178.

In [32]:
# benchmark time it takes to do a timestep on the CPU
moveArrays!(mpasOcean, Array)
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark forward_backward_step!(mpasOcean)

BechmarkTools.Trial: 250 samples with 1 evaluations.
 Range (min … max):  17.490 ms … 29.932 ms  ┊ GC (min … max): 0.00% … 15.09%
 Time  (median):     18.887 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   20.012 ms ±  2.178 ms  ┊ GC (mean ± σ):  6.55% ±  8.76%

        ▄█                                                     
  ▃▄▆▅▅▆██▆▃▂▃▁▂▁▂▂▁▁▁▁▁▂▃▅▄▄▄▆▅▃▃▁▁▂▃▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▃
  17.5 ms         Histogram: frequency by time        28.4 ms <

 Memory estimate: 17.05 MiB, allocs estimate: 955191.

In [20]:
"CUDA kernel evaluation is $((sum(cpubench.times)/length(cpubench.times)) / (sum(gpubench.times)/length(gpubench.times))) times faster than the CPU evaluation on average"

"CUDA kernel evaluation is 463.5924088194348 times faster than the CPU evaluation on average"

### normal velocty tendency calculation only

In [21]:
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark calculate_normal_velocity_tendency_cuda!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):   4.501 μs … 743.442 μs  ┊ GC (min … max): 0.00% … 97.60%
 Time  (median):     20.223 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   20.198 μs ±  15.009 μs  ┊ GC (mean ± σ):  1.43% ±  1.95%

  ▃                                                   ▄█       ▁
  ██▄▃▁▁▄▄▄▁▄▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▃▁▁▁▃▁▁▁▁▃▄▄▅▇▇▆▆▆▆██▆▆▆▆▆▇ █
  4.5 μs        Histogram: log(frequency) by time      22.2 μs <

 Memory estimate: 3.08 KiB, allocs estimate: 57.

In [22]:
moveArrays!(mpasOcean, Array)
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark calculate_normal_velocity_tendency!(mpasOcean)

BechmarkTools.Trial: 307 samples with 1 evaluations.
 Range (min … max):  14.158 ms … 21.202 ms  ┊ GC (min … max): 0.00% … 19.17%
 Time  (median):     15.531 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.294 ms ±  1.730 ms  ┊ GC (mean ± σ):  6.43% ±  9.40%

          ▁█  ▄▆                                               
  ▄▃▄▃▅▄▄▆██▅████▆▅▅▁▂▂▂▂▂▁▁▁▁▂▁▂▁▁▁▁▁▁▃▃▃▃▄▃▄▃▆▅▅▃▄▂▃▂▂▄▄▂▂▂ ▃
  14.2 ms         Histogram: frequency by time        20.3 ms <

 Memory estimate: 13.24 MiB, allocs estimate: 725293.

In [23]:
"CUDA kernel evaluation is $((sum(cpubench.times)/length(cpubench.times)) / (sum(gpubench.times)/length(gpubench.times))) times faster than the CPU evaluation on average"

"CUDA kernel evaluation is 806.7136301657602 times faster than the CPU evaluation on average"

### experimenting with making method more effecient

In [25]:
function calculate_normal_velocity_tendency_cuda2!(mpasOcean::MPAS_Ocean)
    
    CUDA.@cuda blocks=cld(mpasOcean.nEdges, 1024) threads=1024 maxregs=64 calculate_normal_velocity_tendency_cuda_kernel2!(
                                                                        mpasOcean.nEdges,
                                                                        mpasOcean.normalVelocityTendency,
                                                                        mpasOcean.normalVelocityCurrent,
                                                                        mpasOcean.sshCurrent,
                                                                        mpasOcean.cellsOnEdge,
                                                                        mpasOcean.nEdgesOnEdge,
                                                                        mpasOcean.edgesOnEdge,
                                                                        mpasOcean.weightsOnEdge,
                                                                        mpasOcean.fEdge,
                                                                        mpasOcean.dcEdge,
                                                                        mpasOcean.gravity)
end

calculate_normal_velocity_tendency_cuda2! (generic function with 1 method)

In [26]:
function calculate_normal_velocity_tendency_cuda_kernel2!(nEdges,
                                                          normalVelocityTendency,
                                                          normalVelocity,
                                                          ssh,
                                                          cellsOnEdge,
                                                          nEdgesOnEdge,
                                                          edgesOnEdge,
                                                          weightsOnEdge,
                                                          fEdge,
                                                          dcEdge,
                                                          gravity)
    
    iEdge = (CUDA.blockIdx().x - 1) * CUDA.blockDim().x + CUDA.threadIdx().x
    if iEdge <= nEdges
        # read global memory into thread registers
        cell1 = cellsOnEdge[1,iEdge]
        ssh1 = ssh[cell1]
        cell2 = cellsOnEdge[2,iEdge]
        ssh2 = ssh[cell2]
        dc = dcEdge[iEdge]
        neoe = nEdgesOnEdge[iEdge]
        
#         eoes = edgesOnEdge[iEdge]
        
        
        
        # gravity term: take gradient of ssh across edge
        tendency = gravity * ( ssh1 - ssh2 ) / dc

        # coriolis term: combine norm. vel. of surrounding edges to approx. tangential vel.
        for i = 1:neoe
            eoe = edgesOnEdge[i,iEdge]
            tendency += weightsOnEdge[i,iEdge] * normalVelocity[eoe] * fEdge[eoe]
        end

        normalVelocityTendency[iEdge] = tendency
    end
    return
end

calculate_normal_velocity_tendency_cuda_kernel2! (generic function with 1 method)

In [ ]:
function calculate_ssh_tendency_cuda_kernel2!(nCells,
                                              sshTendency,
                                              ssh,
                                              normalVelocity,
                                              bottomDepth,
                                              nEdgesOnCell,
                                              edgesOnCell,
                                              cellsOnCell,
                                              areaCell,
                                              edgeSignOnCell,
                                              dvEdge)
    
    iCell = (CUDA.blockIdx().x - 1) * CUDA.blockDim().x + CUDA.threadIdx().x
    if iCell <= nCells
        
        sshTendency[iCell] = 0
        
        # sum flux through each edge of cell
        for i in 1:nEdgesOnCell[iCell]
            edgeID = edgesOnCell[i,iCell]
            neighborCellID = cellsOnCell[i,iCell]

            if neighborCellID !== 0
                depth = ( bottomDepth[neighborCellID] + bottomDepth[iCell] ) / 2.0
            else
                depth = bottomDepth[iCell]
            end
            
            flux = depth * normalVelocity[edgeID]
            sshTendency[iCell] += flux * edgeSignOnCell[iCell,i] * dvEdge[edgeID] / areaCell[iCell]
        end
        
    end
    return
end


In [27]:
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark calculate_normal_velocity_tendency_cuda2!(mpasOcean)

LoadError: GPU compilation of kernel calculate_normal_velocity_tendency_cuda_kernel2!(Int64, CUDA.CuDeviceVector{Float64, 1}, CUDA.CuDeviceVector{Float64, 1}, CUDA.CuDeviceVector{Float64, 1}, CUDA.CuDeviceMatrix{Int32, 1}, CUDA.CuDeviceVector{Int32, 1}, Vector{Any}, CUDA.CuDeviceMatrix{Float64, 1}, CUDA.CuDeviceVector{Float64, 1}, CUDA.CuDeviceVector{Float64, 1}, Float64) failed
KernelError: passing and using non-bitstype argument

Argument 8 to your kernel function is of type Vector{Any}, which is not isbits:



### ssh tendency calculation only

In [25]:
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark calculate_ssh_tendency_cuda!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):   4.374 μs … 760.519 μs  ┊ GC (min … max): 0.00% … 96.73%
 Time  (median):     27.183 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   27.210 μs ±  14.107 μs  ┊ GC (mean ± σ):  0.81% ±  1.69%

  ▃                         █                                  ▁
  █▄▄▃▁▃▃▁▁▁▁▃▁▁▁▁▁▁▃▃▁▃▄▇████▇▄▄▁▁▁▁▁▁▁▁▁▄▁▁▃▄▄▃▁▁▁▁▁▁▄▁▅▆▄▃▆ █
  4.37 μs       Histogram: log(frequency) by time      56.2 μs <

 Memory estimate: 3.11 KiB, allocs estimate: 55.

In [26]:
moveArrays!(mpasOcean, Array)
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark calculate_ssh_tendency!(mpasOcean)

BechmarkTools.Trial: 1283 samples with 1 evaluations.
 Range (min … max):  3.539 ms …   7.596 ms  ┊ GC (min … max): 0.00% … 41.13%
 Time  (median):     3.672 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.895 ms ± 746.883 μs  ┊ GC (mean ± σ):  5.31% ± 11.34%

  ▂▆█▄▄▃                                                ▁      
  ██████▇▅▆▆▁▁▄▄▄▄▄▁▁▅▆▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▇███▇▇▆ █
  3.54 ms      Histogram: log(frequency) by time      6.69 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 229892.

In [27]:
"CUDA kernel evaluation is $((sum(cpubench.times)/length(cpubench.times)) / (sum(gpubench.times)/length(gpubench.times))) times faster than the CPU evaluation on average"

"CUDA kernel evaluation is 143.1461720325683 times faster than the CPU evaluation on average"

### update norm vel

In [28]:
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark update_normal_velocity_by_tendency_cuda!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.152 μs … 735.304 μs  ┊ GC (min … max): 0.00% … 99.27%
 Time  (median):     3.337 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.535 μs ±  10.279 μs  ┊ GC (mean ± σ):  4.10% ±  1.40%

     ██▇▅▁                                                     
  ▂▂▆█████▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂ ▃
  3.15 μs         Histogram: frequency by time        5.14 μs <

 Memory estimate: 1.39 KiB, allocs estimate: 33.

In [29]:
moveArrays!(mpasOcean, Array)
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark update_normal_velocity_by_tendency!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.454 μs …   5.509 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.545 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.580 μs ± 286.345 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃▂█▁                                                        
  ▆████▄▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▂
  1.45 μs         Histogram: frequency by time        3.14 μs <

 Memory estimate: 80 bytes, allocs estimate: 3.

In [30]:
"CUDA kernel evaluation is $((sum(cpubench.times)/length(cpubench.times)) / (sum(gpubench.times)/length(gpubench.times))) times faster than the CPU evaluation on average"

"CUDA kernel evaluation is 0.44711592195483957 times faster than the CPU evaluation on average"

### update ssh

In [31]:
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark update_ssh_by_tendency_cuda!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.021 μs … 729.140 μs  ┊ GC (min … max): 0.00% … 99.33%
 Time  (median):     3.220 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.348 μs ±   7.266 μs  ┊ GC (mean ± σ):  2.16% ±  0.99%

     ▃██▆▄▁                                                    
  ▂▃▆██████▆▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂ ▃
  320 μs          Histogram: frequency by time        4.94 μs <

 Memory estimate: 1.39 KiB, allocs estimate: 33.

In [32]:
moveArrays!(mpasOcean, Array)
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark update_ssh_by_tendency!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 162 evaluations.
 Range (min … max):  664.253 ns …  27.238 μs  ┊ GC (min … max): 0.00% … 97.05%
 Time  (median):     713.883 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   722.487 ns ± 371.018 ns  ┊ GC (mean ± σ):  0.72% ±  1.37%

           ▁▂▃▄█▇▃▂▁                                             
  ▂▂▃▄▄▆█▇▇█████████▇▇▆▅▄▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂ ▃
  664 ns           Histogram: frequency by time          885 ns <

 Memory estimate: 80 bytes, allocs estimate: 3.

In [33]:
"CUDA kernel evaluation is $((sum(cpubench.times)/length(cpubench.times)) / (sum(gpubench.times)/length(gpubench.times))) times faster than the CPU evaluation on average"

"CUDA kernel evaluation is 0.21581601461375574 times faster than the CPU evaluation on average"